# Modeling

In [1]:
import numpy as np
import pandas as pd

import unicodedata

import re

import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

import acquire as ac
import prepare as pr
import preprocessing as pp
import evaluate as ev

---
## Wrangle

In [2]:
df = pr.wrangle_readme_data()
df

,repo,language,readme_contents,clean_readme_contents,len_of_clean_readme_contents
0,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction second programming assignment req...,353
1,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello repository meant provide example fo...,76
2,tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www....",div align center src http www tensorflow org i...,1461
3,SmartThingsCommunity/SmartThingsPublic,Groovy,# SmartThings Public GitHub Repo\n\nAn officia...,smartthings public github repo official list s...,62
4,twbs/bootstrap,JavaScript,"<p align=""center"">\n <a href=""https://getboot...",p align center href http getbootstrap com src ...,1399
...,...,...,...,...,...
211,akveo/ngx-admin,TypeScript,"# ngx-admin [<img src=""https://i.imgur.com/oMc...",ngx admin src http imgur com omcxwz0 png alt e...,538
212,swirldev/swirl_courses,R,# swirl courses\n\nThis is a collection of int...,swirl course collection interactive course use...,491
213,jrowberg/i2cdevlib,C++,Jennic platform added!\n\n====================...,jennic platform added note detail project plea...,271
214,etcd-io/etcd,Go,# etcd\n\n[![Go Report Card](https://goreportc...,etcd go report card http goreportcard com badg...,1135


In [3]:
df.columns

Index(['repo', 'language', 'readme_contents', 'clean_readme_contents',
       'len_of_clean_readme_contents'],
      dtype='object')

In [4]:
df.repo.value_counts().head(18)

crossoverJie/JCSprout                                                          2
fengdu78/Coursera-ML-AndrewNg-Notes                                            2
CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers    2
rdpeng/ProgrammingAssignment2                                                  2
axios/axios                                                                    2
SmartThingsCommunity/SmartThingsPublic                                         2
octocat/Spoon-Knife                                                            2
apache/incubator-mxnet                                                         2
tensorflow/models                                                              2
tensorflow/tensorflow                                                          2
rust-lang/rust                                                                 2
woocommerce/woocommerce                                                        2
arduino/Arduino             

In [5]:
df.repo.value_counts().tail()

ytdl-org/youtube-dl                        1
angular/angular-cli                        1
sahat/hackathon-starter                    1
jackfrued/Python-100-Days                  1
Pierian-Data/Complete-Python-3-Bootcamp    1
Name: repo, dtype: int64

In [6]:
len(df.repo.unique())

199

In [7]:
# stratification base on language using train_test_split won't work unless we have more than one observation
# per language
df.language.value_counts()

JavaScript          46
Java                35
Python              29
C++                 18
HTML                16
Jupyter Notebook    10
PHP                  9
Go                   9
Ruby                 7
TypeScript           7
CSS                  6
C                    5
Vue                  3
R                    3
Rust                 3
C#                   2
Kotlin               2
PowerShell           2
Shell                2
Groovy               2
Name: language, dtype: int64

---
## Model

---
### Outlier Detour

In [8]:
# lemmas
# list_of_readmes = df.clean_readme_contents.tolist()
# list_of_readmes

In [9]:
# for index in range(len(list_of_readmes)):
#     list_of_readmes[index] = list_of_readmes[index].split()
    

# print(list_of_readmes)

In [10]:
# list_of_list_of_lemmas = [lemma.split() for lemmas in list_of_lemmas]
# list_of_list_of_lemmas

In [11]:
# len(list_of_readmes[0])

In [12]:
# len(list_of_readmes[-1])

In [13]:
# len_of_readmes = [len(readme) for readme in list_of_readmes]
# len_of_readmes

In [14]:
# len_of_readmes.sort()

In [15]:
# print(len_of_readmes)

**Cutoff for length of words in readme = 10**

In [16]:
index_three_list = df[df.index == 3].clean_readme_contents.values.tolist()
index_three_list

['smartthings public github repo official list smartapps device type smartthings link help get started coding right away github specific documentation http doc smartthings com en latest tool ide github integration html full documentation http doc smartthings com ide simulator http ide smartthings com community forum http community smartthings com follow u web twitter http twitter com smartthingsdev facebook http facebook com smartthingsdevelopers']

In [17]:
df[df.index == 3].len_of_clean_readme_contents

3    62
Name: len_of_clean_readme_contents, dtype: int64

**len_of_clean_readme_contents seems to be working**

In [18]:
df.len_of_clean_readme_contents.value_counts().sort_index()

10       1
13       2
14       1
26       1
30       1
        ..
9774     1
9792     1
12699    1
17748    1
29238    1
Name: len_of_clean_readme_contents, Length: 184, dtype: int64

In [19]:
df.shape

(216, 5)

**225 observations before outlier removal; should have 219 after**

**CHECK**

---
### Back on Model Track

#### Train/Test Split

In [20]:
df = pr.wrangle_readme_data()
print(df.shape)
df.head()

(216, 5)


,repo,language,readme_contents,clean_readme_contents,len_of_clean_readme_contents
0,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction second programming assignment req...,353
1,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello repository meant provide example fo...,76
2,tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www....",div align center src http www tensorflow org i...,1461
3,SmartThingsCommunity/SmartThingsPublic,Groovy,# SmartThings Public GitHub Repo\n\nAn officia...,smartthings public github repo official list s...,62
4,twbs/bootstrap,JavaScript,"<p align=""center"">\n <a href=""https://getboot...",p align center href http getbootstrap com src ...,1399


## Modeling

Transform your documents into a form that can be used in a machine learning model. You should use the programming language of the repository as the label to predict.

Try fitting several different models and using several different representations of the text (e.g. a simple bag of words, then also the TF-IDF values for each).

Build a function that will take in the text of a README file, and tries to predict the programming language.

**CountVectorizer**

In [21]:
# create cv object
cv = CountVectorizer()

In [22]:
# fit and use the cv object
cv_bag_of_words = cv.fit_transform(df.clean_readme_contents)

In [23]:
cv_bag_of_words

<216x23425 sparse matrix of type '<class 'numpy.int64'>'
	with 76742 stored elements in Compressed Sparse Row format>

**TF-IDF**

In [24]:
# create tfidf vectorizer object
tfidf = TfidfVectorizer()

In [25]:
tfidf_bag_of_words = tfidf.fit_transform(df.clean_readme_contents)

In [26]:
tfidf_bag_of_words

<216x23425 sparse matrix of type '<class 'numpy.float64'>'
	with 76742 stored elements in Compressed Sparse Row format>

**CountVectorizer Bag of Bigrams**

In [27]:
# create cv for bigrams
cv_bigrams = CountVectorizer(ngram_range=(2, 2))

In [28]:
cv_bag_of_bigrams = cv_bigrams.fit_transform(df.clean_readme_contents)

In [29]:
cv_bag_of_bigrams

<216x142894 sparse matrix of type '<class 'numpy.int64'>'
	with 181901 stored elements in Compressed Sparse Row format>

**TF-IDF Bag of Bigrams**

In [30]:
tfidf_bigrams = TfidfVectorizer(ngram_range=(2, 2))

In [31]:
tfidf_bag_of_bigrams = tfidf_bigrams.fit_transform(df.clean_readme_contents)

In [32]:
tfidf_bag_of_bigrams

<216x142894 sparse matrix of type '<class 'numpy.float64'>'
	with 181901 stored elements in Compressed Sparse Row format>

___

**Model #1: Standard CV**

In [33]:
# establish features and target for CountVectorizer model
X = cv.fit_transform(df.clean_readme_contents)
y = df.language

In [34]:
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

In [35]:
print(X_train.shape)
print(y_train.shape)
print(X_train.shape[0]/df.shape[0])
print(y_train.shape[0]/df.shape[0])

(172, 23425)
(172,)
0.7962962962962963
0.7962962962962963


In [36]:
print(X_test.shape)
print(y_test.shape)
print(X_test.shape[0]/df.shape[0])
print(y_test.shape[0]/df.shape[0])

(44, 23425)
(44,)
0.2037037037037037
0.2037037037037037


In [37]:
pd.DataFrame(X_train[:5, :].todense(), columns=cv.get_feature_names())

,00,000,000000,00042,001,0011345406,002,003,00536,0059b3,...,zxing,zxingobjc,zxingorg,zygmuntz,zyiot,zyiz,zynga,zypper,zyszys,zyszys98
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# create evaluation DataFrame
evaluation = pd.DataFrame(columns=["model_type", "accuracy"])

In [39]:
# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = tree.predict(X_train)
test['predicted'] = tree.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="CV Gini, 5", model_object=tree, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
0,"CV Gini, 5",0.569767


In [40]:
print('CV DT Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

CV DT Model Train Accuracy: 56.98%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,1.000000,0.0,0.833333,0.0,0.174419,1.000000,0.965517,1.000000,...,1.000000,0.0,0.0,0.0,0.0,1.000000,1.0,0.569767,0.423663,0.721539
recall,0.0,0.0,0.307692,0.0,0.714286,0.0,1.000000,0.800000,0.717949,0.700000,...,0.526316,0.0,0.0,0.0,0.0,0.714286,1.0,0.569767,0.349026,0.569767
f1-score,0.0,0.0,0.470588,0.0,0.769231,0.0,0.297030,0.888889,0.823529,0.823529,...,0.689655,0.0,0.0,0.0,0.0,0.833333,1.0,0.569767,0.354789,0.589940
support,3.0,2.0,13.000000,3.0,7.000000,2.0,15.000000,25.000000,39.000000,10.000000,...,19.000000,2.0,7.0,2.0,2.0,7.000000,3.0,0.569767,172.000000,172.000000


In [41]:
print('CV DT Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

CV DT Model Test Accuracy: 34.09%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C++,CSS,Go,HTML,Java,JavaScript,Kotlin,PHP,Python,R,Rust,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,1.0,0.0,0.875000,0.833333,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.340909,0.211310,0.433712
recall,0.0,0.0,0.0,1.0,0.0,0.700000,0.714286,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.340909,0.179592,0.340909
f1-score,0.0,0.0,0.0,1.0,0.0,0.777778,0.769231,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.340909,0.192133,0.377067
support,2.0,5.0,3.0,2.0,1.0,10.000000,7.000000,0.0,2.0,10.000000,1.0,1.0,0.0,0.0,0.340909,44.000000,44.000000


**Model #2: TF-IDF**

In [42]:
# establish features and target for tfidf model

# create tfidf vectorizer object
tfidf = TfidfVectorizer()

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = tree.predict(X_train)
test['predicted'] = tree.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Gini, 5", model_object=tree, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
1,"TF-IDF Gini, 5",0.587209
0,"CV Gini, 5",0.569767


In [43]:
print('TF-IDF DT Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

TF-IDF DT Model Train Accuracy: 58.72%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,1.000000,0.0,1.000000,0.0,0.180723,1.000000,1.000000,1.000000,...,0.812500,0.0,0.0,0.0,0.0,1.000000,1.0,0.587209,0.399661,0.710165
recall,0.0,0.0,0.307692,0.0,0.714286,0.0,1.000000,0.800000,0.692308,0.900000,...,0.684211,0.0,0.0,0.0,0.0,0.714286,1.0,0.587209,0.340639,0.587209
f1-score,0.0,0.0,0.470588,0.0,0.833333,0.0,0.306122,0.888889,0.818182,0.947368,...,0.742857,0.0,0.0,0.0,0.0,0.833333,1.0,0.587209,0.342034,0.599392
support,3.0,2.0,13.000000,3.0,7.000000,2.0,15.000000,25.000000,39.000000,10.000000,...,19.000000,2.0,7.0,2.0,2.0,7.000000,3.0,0.587209,172.000000,172.000000


In [44]:
print('TF-IDF DT Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

TF-IDF DT Model Test Accuracy: 36.36%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C++,CSS,Go,HTML,Java,JavaScript,Jupyter Notebook,PHP,Python,R,Rust,TypeScript,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,1.000000,0.833333,0.0,0.0,0.571429,0.0,0.0,0.0,0.363636,0.184982,0.489719
recall,0.0,0.0,0.0,0.0,0.0,0.700000,0.714286,0.0,0.0,0.400000,0.0,0.0,0.0,0.363636,0.139560,0.363636
f1-score,0.0,0.0,0.0,0.0,0.0,0.823529,0.769231,0.0,0.0,0.470588,0.0,0.0,0.0,0.363636,0.158719,0.416495
support,2.0,5.0,3.0,2.0,1.0,10.000000,7.000000,0.0,2.0,10.000000,1.0,1.0,0.0,0.363636,44.000000,44.000000


**Model #3: CV Bigrams**

In [45]:
# establish features and target for cv model

# create cv_bigrams vectorizer object
cv_bigrams = CountVectorizer(ngram_range=(2, 2))

# use tfidf object to create model features
X = cv_bigrams.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = tree.predict(X_train)
test['predicted'] = tree.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="CV Bigrams Gini, 5", model_object=tree, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
1,"TF-IDF Gini, 5",0.587209
0,"CV Gini, 5",0.569767
2,"CV Bigrams Gini, 5",0.412791


In [46]:
print('CV Bigrams Gini, 5 Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

CV Bigrams Gini, 5 Model Train Accuracy: 41.28%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,1.000000,0.0,0.500000,0.0,0.140187,0.923077,0.793103,0.875000,...,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.412791,0.311568,0.624190
recall,0.0,0.0,0.153846,0.0,0.142857,0.0,1.000000,0.480000,0.589744,0.700000,...,0.526316,0.0,0.0,0.0,0.0,0.142857,0.0,0.412791,0.186781,0.412791
f1-score,0.0,0.0,0.266667,0.0,0.222222,0.0,0.245902,0.631579,0.676471,0.777778,...,0.689655,0.0,0.0,0.0,0.0,0.250000,0.0,0.412791,0.188014,0.427406
support,3.0,2.0,13.000000,3.0,7.000000,2.0,15.000000,25.000000,39.000000,10.000000,...,19.000000,2.0,7.0,2.0,2.0,7.000000,3.0,0.412791,172.000000,172.000000


In [47]:
print('CV Bigrams Gini, 5 Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

CV Bigrams Gini, 5 Model Test Accuracy: 22.73%
---


,C,C++,CSS,Go,HTML,Java,JavaScript,PHP,Python,R,Rust,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.035714,0.333333,0.714286,0.0,0.500,0.0,0.0,0.227273,0.143939,0.303842
recall,0.0,0.0,0.0,0.0,1.000000,0.100000,0.714286,0.0,0.300,0.0,0.0,0.227273,0.192208,0.227273
f1-score,0.0,0.0,0.0,0.0,0.068966,0.153846,0.714286,0.0,0.375,0.0,0.0,0.227273,0.119282,0.235396
support,2.0,5.0,3.0,2.0,1.000000,10.000000,7.000000,2.0,10.000,1.0,1.0,0.227273,44.000000,44.000000


**Model #4: TF-IDF Bigrams**

In [48]:
# establish features and target for tfidf model

# create cv_bigrams vectorizer object
tfidf_bigrams = TfidfVectorizer(ngram_range=(2, 2))

# use tfidf object to create model features
X = tfidf_bigrams.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = tree.predict(X_train)
test['predicted'] = tree.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Bigrams Gini, 5", model_object=tree, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
1,"TF-IDF Gini, 5",0.587209
0,"CV Gini, 5",0.569767
3,"TF-IDF Bigrams Gini, 5",0.447674
2,"CV Bigrams Gini, 5",0.412791


In [49]:
print('TF-IDF Bigrams Gini, 5 Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

TF-IDF Bigrams Gini, 5 Model Train Accuracy: 44.77%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,1.000000,0.0,1.000000,0.0,0.0,1.000000,0.287879,0.875000,...,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.447674,0.358144,0.540566
recall,0.0,0.0,0.153846,0.0,0.714286,0.0,0.0,0.480000,0.974359,0.700000,...,0.578947,0.0,0.0,0.0,0.0,0.142857,0.0,0.447674,0.212215,0.447674
f1-score,0.0,0.0,0.266667,0.0,0.833333,0.0,0.0,0.648649,0.444444,0.777778,...,0.733333,0.0,0.0,0.0,0.0,0.250000,0.0,0.447674,0.231044,0.393279
support,3.0,2.0,13.000000,3.0,7.000000,2.0,15.0,25.000000,39.000000,10.000000,...,19.000000,2.0,7.0,2.0,2.0,7.000000,3.0,0.447674,172.000000,172.000000


In [50]:
print('TF-IDF Bigrams Gini, 5 Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

TF-IDF Bigrams Gini, 5 Model Test Accuracy: 25.00%
---


,C,C++,CSS,Go,HTML,Java,JavaScript,PHP,Python,R,Rust,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,0.333333,0.200000,0.0,0.500,0.0,0.0,0.25,0.093939,0.221212
recall,0.0,0.0,0.0,0.0,0.0,0.100000,1.000000,0.0,0.300,0.0,0.0,0.25,0.127273,0.250000
f1-score,0.0,0.0,0.0,0.0,0.0,0.153846,0.333333,0.0,0.375,0.0,0.0,0.25,0.078380,0.173223
support,2.0,5.0,3.0,2.0,1.0,10.000000,7.000000,2.0,10.000,1.0,1.0,0.25,44.000000,44.000000


**Model #5: TF-IDF Unigrams & Bigrams**

In [51]:
# establish features and target for tfidf model

# create cv_bigrams vectorizer object
tfidf_unigrams_and_bigrams = TfidfVectorizer(ngram_range=(1, 2))

# use tfidf object to create model features
X = tfidf_unigrams_and_bigrams.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
tree = DecisionTreeClassifier(max_depth=5, random_state=56)

# fit model object
tree.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = tree.predict(X_train)
test['predicted'] = tree.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Unigrams & Bigrams Gini, 5", model_object=tree, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
1,"TF-IDF Gini, 5",0.587209
4,"TF-IDF Unigrams & Bigrams Gini, 5",0.581395
0,"CV Gini, 5",0.569767
3,"TF-IDF Bigrams Gini, 5",0.447674
2,"CV Bigrams Gini, 5",0.412791


In [52]:
print('TF-IDF Unigrams & Bigrams Gini, 5 Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

TF-IDF Unigrams & Bigrams Gini, 5 Model Train Accuracy: 58.14%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,1.000000,0.0,1.000000,0.0,0.178571,1.000000,1.000000,1.000000,...,0.812500,0.0,0.0,0.0,0.0,1.000000,1.0,0.581395,0.449554,0.721605
recall,0.0,0.0,0.307692,0.0,0.714286,0.0,1.000000,0.800000,0.692308,0.800000,...,0.684211,0.0,0.0,0.0,0.0,0.571429,1.0,0.581395,0.353496,0.581395
f1-score,0.0,0.0,0.470588,0.0,0.833333,0.0,0.303030,0.888889,0.818182,0.888889,...,0.742857,0.0,0.0,0.0,0.0,0.727273,1.0,0.581395,0.366985,0.599158
support,3.0,2.0,13.000000,3.0,7.000000,2.0,15.000000,25.000000,39.000000,10.000000,...,19.000000,2.0,7.0,2.0,2.0,7.000000,3.0,0.581395,172.000000,172.000000


In [53]:
print('TF-IDF Unigrams & Bigrams Gini, 5 Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

TF-IDF Unigrams & Bigrams Gini, 5 Model Test Accuracy: 25.00%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C++,CSS,Go,HTML,Java,JavaScript,Jupyter Notebook,PHP,Python,R,Rust,TypeScript,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,0.6,0.833333,0.0,0.0,0.3,0.0,0.0,0.0,0.25,0.133333,0.337121
recall,0.0,0.0,0.0,0.0,0.0,0.3,0.714286,0.0,0.0,0.3,0.0,0.0,0.0,0.25,0.101099,0.250000
f1-score,0.0,0.0,0.0,0.0,0.0,0.4,0.769231,0.0,0.0,0.3,0.0,0.0,0.0,0.25,0.113018,0.281469
support,2.0,5.0,3.0,2.0,1.0,10.0,7.000000,0.0,2.0,10.0,1.0,1.0,0.0,0.25,44.000000,44.000000


---

**Create a function to append evaluation metrics (accuracy) to evaluation dataframe**


**CHECK**

---

**Model #6: TF-IDF Logistic Regression**

In [54]:
# establish features and target for tfidf logit model

# create cv_bigrams vectorizer object
tfidf = TfidfVectorizer()

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
logit = LogisticRegression(random_state=56)

# fit model object
logit.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = logit.predict(X_train)
test['predicted'] = logit.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Logistic Regression", model_object=logit, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
5,TF-IDF Logistic Regression,0.645349
1,"TF-IDF Gini, 5",0.587209
4,"TF-IDF Unigrams & Bigrams Gini, 5",0.581395
0,"CV Gini, 5",0.569767
3,"TF-IDF Bigrams Gini, 5",0.447674
2,"CV Bigrams Gini, 5",0.412791


In [55]:
print('TF-IDF Logistic Regression Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

TF-IDF Logistic Regression Model Train Accuracy: 64.53%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,1.000000,0.0,0.0,0.0,1.000000,0.925926,0.397959,1.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.645349,0.316194,0.596910
recall,0.0,0.0,0.538462,0.0,0.0,0.0,0.666667,1.000000,1.000000,0.700000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.645349,0.273828,0.645349
f1-score,0.0,0.0,0.700000,0.0,0.0,0.0,0.800000,0.961538,0.569343,0.823529,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.645349,0.279084,0.579471
support,3.0,2.0,13.000000,3.0,7.0,2.0,15.000000,25.000000,39.000000,10.000000,...,19.0,2.0,7.0,2.0,2.0,7.0,3.0,0.645349,172.000000,172.000000


In [56]:
print('TF-IDF Logistic Regression Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

TF-IDF Logistic Regression Model Test Accuracy: 36.36%
---


,C,C++,CSS,Go,HTML,Java,JavaScript,PHP,Python,R,Rust,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,1.000000,0.212121,0.0,0.666667,0.0,0.0,0.363636,0.170799,0.412534
recall,0.0,0.0,0.0,0.0,0.0,0.500000,1.000000,0.0,0.400000,0.0,0.0,0.363636,0.172727,0.363636
f1-score,0.0,0.0,0.0,0.0,0.0,0.666667,0.350000,0.0,0.500000,0.0,0.0,0.363636,0.137879,0.320833
support,2.0,5.0,3.0,2.0,1.0,10.000000,7.000000,2.0,10.000000,1.0,1.0,0.363636,44.000000,44.000000


In [57]:
# go_df = df[df.language == "Go"]
# go_df.to_dict("response")

**Model #7: TF-IDF Hyperparameters**

In [58]:
# establish features and target for tfidf model

# create tfidf vectorizer object
tfidf = TfidfVectorizer()

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y, train_size=.7, test_size=.3)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
tree = DecisionTreeClassifier(criterion="entropy", max_depth=5, min_samples_leaf=2,random_state=56)

# fit model object
tree.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = tree.predict(X_train)
test['predicted'] = tree.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Entropy, 5, 2, Train 70%", model_object=tree, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
6,"TF-IDF Entropy, 5, 2, Train 70%",0.662252
5,TF-IDF Logistic Regression,0.645349
1,"TF-IDF Gini, 5",0.587209
4,"TF-IDF Unigrams & Bigrams Gini, 5",0.581395
0,"CV Gini, 5",0.569767
3,"TF-IDF Bigrams Gini, 5",0.447674
2,"CV Bigrams Gini, 5",0.412791


In [59]:
print('TF-IDF Entropy, 5, 2, Train 70% Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

TF-IDF Entropy, 5, 2, Train 70% Model Train Accuracy: 66.23%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.354839,0.0,0.0,0.0,0.476190,1.000000,0.727273,1.0,...,1.00,0.0,0.0,0.0,0.0,1.000000,1.0,0.662252,0.373748,0.627787
recall,0.0,0.0,0.846154,0.0,0.0,0.0,0.769231,0.782609,0.864865,1.0,...,0.60,0.0,0.0,0.0,0.0,0.500000,1.0,0.662252,0.393143,0.662252
f1-score,0.0,0.0,0.500000,0.0,0.0,0.0,0.588235,0.878049,0.790123,1.0,...,0.75,0.0,0.0,0.0,0.0,0.666667,1.0,0.662252,0.363065,0.618772
support,3.0,2.0,13.000000,3.0,7.0,1.0,13.000000,23.000000,37.000000,9.0,...,15.00,1.0,7.0,1.0,1.0,4.000000,3.0,0.662252,151.000000,151.000000


In [60]:
print('TF-IDF Entropy, 5, 2, Train 70% Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

TF-IDF Entropy, 5, 2, Train 70% Model Test Accuracy: 32.31%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,Kotlin,...,PowerShell,Python,R,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,0.083333,0.857143,0.545455,0.0,0.0,...,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.323077,0.160329,0.477612
recall,0.0,0.0,0.0,0.0,0.0,0.333333,0.500000,0.666667,0.0,0.0,...,0.0,0.428571,0.0,0.0,0.0,0.0,0.0,0.323077,0.134921,0.323077
f1-score,0.0,0.0,0.0,0.0,0.0,0.133333,0.631579,0.600000,0.0,0.0,...,0.0,0.600000,0.0,0.0,0.0,0.0,0.0,0.323077,0.133853,0.362411
support,2.0,5.0,3.0,2.0,1.0,3.000000,12.000000,9.000000,1.0,0.0,...,1.0,14.000000,2.0,2.0,1.0,3.0,0.0,0.323077,65.000000,65.000000


In [61]:
pd.Series(dict(zip(tfidf.get_feature_names(), tree.feature_importances_))).sort_values(ascending=False).head(20)

npm           0.202207
python        0.174204
java          0.123546
please        0.100633
using         0.069256
security      0.066815
br            0.053770
build         0.038664
im            0.036539
dependency    0.029792
money         0.027945
graph         0.022417
makefile      0.020491
api           0.014463
issue         0.014463
different     0.004796
ergast        0.000000
erail         0.000000
erased        0.000000
ericchiang    0.000000
dtype: float64

In [62]:
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
6,"TF-IDF Entropy, 5, 2, Train 70%",0.662252
5,TF-IDF Logistic Regression,0.645349
1,"TF-IDF Gini, 5",0.587209
4,"TF-IDF Unigrams & Bigrams Gini, 5",0.581395
0,"CV Gini, 5",0.569767
3,"TF-IDF Bigrams Gini, 5",0.447674
2,"CV Bigrams Gini, 5",0.412791


---
**What words are appearing most frequently depending on language?**

---

### Stratification Detour

In [63]:
# df.shape

In [64]:
# df.language.value_counts() >= 2

In [65]:
# df.groupby('language').filter(lambda x : len(x) >= 2)

In [66]:
# df.groupby("language").language.agg(["count"]).sort_values(by="count") 

In [67]:
# df[df.groupby("language").language.transform("count") >= 2]

In [68]:
# df = df[df.groupby("language").language.transform("count") >= 2]

In [69]:
# df[df.language == "ApacheConf"]

**Model #8: TF-IDF Bigrams Logistic Regression**

In [70]:
# establish features and target for tfidf logit model

# create cv_bigrams vectorizer object
tfidf = TfidfVectorizer(ngram_range=(2, 2))

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
logit = LogisticRegression(random_state=56)

# fit model object
logit.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = logit.predict(X_train)
test['predicted'] = logit.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Bigrams Logistic Regression", model_object=logit, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
6,"TF-IDF Entropy, 5, 2, Train 70%",0.662252
5,TF-IDF Logistic Regression,0.645349
7,TF-IDF Bigrams Logistic Regression,0.604651
1,"TF-IDF Gini, 5",0.587209
4,"TF-IDF Unigrams & Bigrams Gini, 5",0.581395
0,"CV Gini, 5",0.569767
3,"TF-IDF Bigrams Gini, 5",0.447674
2,"CV Bigrams Gini, 5",0.412791


In [71]:
print('TF-IDF Bigrams Logistic Regression Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

TF-IDF Bigrams Logistic Regression Model Train Accuracy: 60.47%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,1.000000,0.0,0.0,0.0,1.000000,1.0,0.364486,1.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604651,0.318224,0.600087
recall,0.0,0.0,0.230769,0.0,0.0,0.0,0.800000,1.0,1.000000,0.200000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604651,0.240110,0.604651
f1-score,0.0,0.0,0.375000,0.0,0.0,0.0,0.888889,1.0,0.534247,0.333333,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604651,0.242937,0.531792
support,3.0,2.0,13.000000,3.0,7.0,2.0,15.000000,25.0,39.000000,10.000000,...,19.0,2.0,7.0,2.0,2.0,7.0,3.0,0.604651,172.000000,172.000000


In [72]:
print('TF-IDF Bigrams Logistic Regression Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

TF-IDF Bigrams Logistic Regression Model Test Accuracy: 20.45%
---


,C,C++,CSS,Go,HTML,Java,JavaScript,PHP,Python,R,Rust,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,1.000000,0.0,0.0,0.204545,0.106061,0.253788
recall,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.200000,0.0,0.0,0.204545,0.109091,0.204545
f1-score,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,0.333333,0.0,0.0,0.204545,0.056277,0.121212
support,2.0,5.0,3.0,2.0,1.0,10.0,7.000000,2.0,10.000000,1.0,1.0,0.204545,44.000000,44.000000


**Model #9: TF-IDF Unigrams & Bigrams Logistics Regression**

In [73]:
# establish features and target for tfidf logit model

# create cv_bigrams vectorizer object
tfidf = TfidfVectorizer(ngram_range=(1, 2))

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
logit = LogisticRegression(random_state=56)

# fit model object
logit.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = logit.predict(X_train)
test['predicted'] = logit.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Unigram & Bigram Logistic Regression", model_object=logit, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
6,"TF-IDF Entropy, 5, 2, Train 70%",0.662252
5,TF-IDF Logistic Regression,0.645349
8,TF-IDF Unigram & Bigram Logistic Regression,0.610465
7,TF-IDF Bigrams Logistic Regression,0.604651
1,"TF-IDF Gini, 5",0.587209
4,"TF-IDF Unigrams & Bigrams Gini, 5",0.581395
0,"CV Gini, 5",0.569767
3,"TF-IDF Bigrams Gini, 5",0.447674
2,"CV Bigrams Gini, 5",0.412791


In [74]:
print('TF-IDF Unigram & Bigram Logistic Regression Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

TF-IDF Unigram & Bigram Logistic Regression Model Train Accuracy: 61.05%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,1.000000,0.0,0.0,0.0,1.00,0.961538,0.371429,1.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.610465,0.316648,0.596071
recall,0.0,0.0,0.307692,0.0,0.0,0.0,0.60,1.000000,1.000000,0.500000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.610465,0.248956,0.610465
f1-score,0.0,0.0,0.470588,0.0,0.0,0.0,0.75,0.980392,0.541667,0.666667,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.610465,0.256829,0.545116
support,3.0,2.0,13.000000,3.0,7.0,2.0,15.00,25.000000,39.000000,10.000000,...,19.0,2.0,7.0,2.0,2.0,7.0,3.0,0.610465,172.000000,172.000000


In [75]:
print('TF-IDF Unigram & Bigram Logistic Regression Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

TF-IDF Unigram & Bigram Logistic Regression Model Test Accuracy: 29.55%
---


,C,C++,CSS,Go,HTML,Java,JavaScript,PHP,Python,R,Rust,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,1.000000,0.184211,0.0,1.000000,0.0,0.0,0.295455,0.198565,0.483852
recall,0.0,0.0,0.0,0.0,0.0,0.300000,1.000000,0.0,0.300000,0.0,0.0,0.295455,0.145455,0.295455
f1-score,0.0,0.0,0.0,0.0,0.0,0.461538,0.311111,0.0,0.461538,0.0,0.0,0.295455,0.112199,0.259285
support,2.0,5.0,3.0,2.0,1.0,10.000000,7.000000,2.0,10.000000,1.0,1.0,0.295455,44.000000,44.000000


**Model #10: Naive Bayes**

In [76]:
# establish features and target for tfidf logit model

# create cv_bigrams vectorizer object
tfidf = TfidfVectorizer()

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
nb = MultinomialNB()

# fit model object
nb.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = nb.predict(X_train)
test['predicted'] = nb.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Naive Bayes", model_object=nb, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
6,"TF-IDF Entropy, 5, 2, Train 70%",0.662252
5,TF-IDF Logistic Regression,0.645349
8,TF-IDF Unigram & Bigram Logistic Regression,0.610465
7,TF-IDF Bigrams Logistic Regression,0.604651
1,"TF-IDF Gini, 5",0.587209
4,"TF-IDF Unigrams & Bigrams Gini, 5",0.581395
0,"CV Gini, 5",0.569767
3,"TF-IDF Bigrams Gini, 5",0.447674
2,"CV Bigrams Gini, 5",0.412791
9,TF-IDF Naive Bayes,0.412791


In [77]:
print('TF-IDF Naive Bayes Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

TF-IDF Naive Bayes Model Train Accuracy: 41.28%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.278571,0.0,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.412791,0.163929,0.406188
recall,0.0,0.0,0.0,0.0,0.0,0.0,0.133333,0.960000,1.000000,0.0,...,0.315789,0.0,0.0,0.0,0.0,0.0,0.0,0.412791,0.120456,0.412791
f1-score,0.0,0.0,0.0,0.0,0.0,0.0,0.235294,0.979592,0.435754,0.0,...,0.480000,0.0,0.0,0.0,0.0,0.0,0.0,0.412791,0.106532,0.314730
support,3.0,2.0,13.0,3.0,7.0,2.0,15.000000,25.000000,39.000000,10.0,...,19.000000,2.0,7.0,2.0,2.0,7.0,3.0,0.412791,172.000000,172.000000


In [78]:
print('TF-IDF Naive Bayes Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

TF-IDF Naive Bayes Model Test Accuracy: 22.73%
---


,C,C++,CSS,Go,HTML,Java,JavaScript,PHP,Python,R,Rust,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,1.000000,0.170732,0.0,1.000000,0.0,0.0,0.227273,0.197339,0.481707
recall,0.0,0.0,0.0,0.0,0.0,0.200000,1.000000,0.0,0.100000,0.0,0.0,0.227273,0.118182,0.227273
f1-score,0.0,0.0,0.0,0.0,0.0,0.333333,0.291667,0.0,0.181818,0.0,0.0,0.227273,0.073347,0.163481
support,2.0,5.0,3.0,2.0,1.0,10.000000,7.000000,2.0,10.000000,1.0,1.0,0.227273,44.000000,44.000000


**Model #11: TF-IDF Bigrams Naive Bayes**

In [79]:
# establish features and target for tfidf logit model

# create cv_bigrams vectorizer object
tfidf = TfidfVectorizer(ngram_range=(2, 2))

# use tfidf object to create model features
X = tfidf.fit_transform(df.clean_readme_contents)

# establish model target
y = df.language

# split data
X_train, X_test, y_train, y_test = pp.split_repo_data(X, y)

# create train and test DataFrames with actual language variable
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# create model object
nb = MultinomialNB()

# fit model object
nb.fit(X_train, y_train)

# create predicted variable in train and test DataFrames
train['predicted'] = nb.predict(X_train)
test['predicted'] = nb.predict(X_test)

# append evaluation
evaluation = ev.append_evaluation(evaluation, model_type="TF-IDF Bigrams Naive Bayes", model_object=nb, X=X_train, y=y_train)
evaluation.sort_values(by="accuracy", ascending=False)

,model_type,accuracy
6,"TF-IDF Entropy, 5, 2, Train 70%",0.662252
5,TF-IDF Logistic Regression,0.645349
8,TF-IDF Unigram & Bigram Logistic Regression,0.610465
7,TF-IDF Bigrams Logistic Regression,0.604651
1,"TF-IDF Gini, 5",0.587209
4,"TF-IDF Unigrams & Bigrams Gini, 5",0.581395
0,"CV Gini, 5",0.569767
10,TF-IDF Bigrams Naive Bayes,0.529070
3,"TF-IDF Bigrams Gini, 5",0.447674
2,"CV Bigrams Gini, 5",0.412791


In [80]:
print('TF-IDF Bigrams Naive Bayes Model Train Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
pd.DataFrame(classification_report(train.actual, train.predicted, output_dict=True))

TF-IDF Bigrams Naive Bayes Model Train Accuracy: 52.91%
---


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,C,C#,C++,CSS,Go,Groovy,HTML,Java,JavaScript,Jupyter Notebook,...,Python,R,Ruby,Rust,Shell,TypeScript,Vue,accuracy,macro avg,weighted avg
precision,0.0,0.0,1.000000,0.0,0.0,0.0,1.000000,1.0,0.325000,1.000000,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.52907,0.266250,0.550436
recall,0.0,0.0,0.153846,0.0,0.0,0.0,0.400000,1.0,1.000000,0.200000,...,0.894737,0.0,0.0,0.0,0.0,0.0,0.0,0.52907,0.182429,0.529070
f1-score,0.0,0.0,0.266667,0.0,0.0,0.0,0.571429,1.0,0.490566,0.333333,...,0.944444,0.0,0.0,0.0,0.0,0.0,0.0,0.52907,0.180322,0.450279
support,3.0,2.0,13.000000,3.0,7.0,2.0,15.000000,25.0,39.000000,10.000000,...,19.000000,2.0,7.0,2.0,2.0,7.0,3.0,0.52907,172.000000,172.000000


In [81]:
print('TF-IDF Bigrams Naive Bayes Model Test Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
pd.DataFrame(classification_report(test.actual, test.predicted, output_dict=True))

TF-IDF Bigrams Naive Bayes Model Test Accuracy: 20.45%
---


,C,C++,CSS,Go,HTML,Java,JavaScript,PHP,Python,R,Rust,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,1.000000,0.0,0.0,0.204545,0.106061,0.253788
recall,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.200000,0.0,0.0,0.204545,0.109091,0.204545
f1-score,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,0.333333,0.0,0.0,0.204545,0.056277,0.121212
support,2.0,5.0,3.0,2.0,1.0,10.0,7.000000,2.0,10.000000,1.0,1.0,0.204545,44.000000,44.000000
